In [1]:
!pip install pymorphy2[fast]

## Преобразователь новостей в токены

In [2]:
from typing import List
import nltk
import string
import pymorphy2
import codecs

class PrepareNew():
  def __init__(self):
    self.morph = pymorphy2.MorphAnalyzer()
    self.tokenizer = nltk.WordPunctTokenizer()
    self.stopwords = set(line.strip() for line in codecs.open('rus_stopwords.txt', "r", "utf_8_sig").readlines())

  def prepare_corp(self, news_list: List[str]):
    return [self.newstext2token(news_text) for news_text in news_list]

  def newstext2token(self, news_text: str):
      tokens = self.tokenizer.tokenize(news_text.lower())
      tokens_with_no_punct = [self.morph.parse(w)[0].normal_form for w in tokens if all(c not in string.punctuation for c in w)]
      tokens_base_forms = [w for w in tokens_with_no_punct if w not in self.stopwords and w.isalpha()]
      tokens_last = [w for w in tokens_base_forms if len(w)>1]
      return tokens_last


## Usage Example

In [3]:
test_news = [
             "Вышла новая серия Локи",
             "Кофе очень вкусный", 
             "Нил Гейман прекрасен",
             "Металлика топ",
             "Толстой тексты, переполненные водой и своеобразным разжёвыванием, повторением информации для читателя"
]
test_corp = PrepareNew().prepare_corp(test_news)
for i in test_corp:
  print(i)

['выйти', 'новый', 'серия', 'лока']
['кофе', 'вкусный']
['нил', 'гейман', 'прекрасный']
['металлик', 'топ']
['толстой', 'текст', 'переполненный', 'вода', 'своеобразный', 'разжёвывание', 'повторение', 'информация', 'читатель']


## Получение текста ведомостей

In [4]:
raw_data = [line.strip() for line in codecs.open('emb_dataset.txt', "r", "utf_8_sig").readlines() if line.strip()!=""]

In [5]:
raw_data[:10]

['На газохранилище в Казани на улице Северо-Западная, 10 произошел взрыв, сообщило МЧС. Один человек погиб, несколько пострадали. Следственный комитет России (СКР) по Татарстану утверждает, что взрыв произошел на газовой заправке. Возбуждено уголовное дело за нарушение правил безопасности на взрывоопасных объектах (ст. 217 УК РФ).',
 '«В результате хлопка, произошедшего на газозаправочной станции, пострадали рабочие, количество которых уточняется. По данному факту следственными органами СКР по республике Татарстан возбуждено уголовное дело»,— говорится в пресс-релизе ведомства.',
 'По данным МЧС, взрыв произошел на наземном газохранилище компании «Газпром сжиженный газ». Пожару присвоен четвертый ранг опасности. Есть риск распространения огня на другие емкости с газом — всего на станции расположено 24 резервуара со сжиженным газом, сообщил источник ТАСС.',
 'Точное число пострадавших неизвестно. Минздрав Татарстана сообщил «РИА Новости» об одном погибшем и двух пострадавших. По данным 

In [6]:
my_corp = PrepareNew().prepare_corp(raw_data)

In [7]:
my_corp = my_corp[1::2]

## Модель-эмбеддер

In [8]:
from gensim.models import Word2Vec
from gensim.models import Phrases
bigram_transformer = Phrases(my_corp)
model = Word2Vec(bigram_transformer[my_corp], window=5, min_count=2)

In [9]:
model.wv.most_similar('рубль', topn=10)

[('миллиард_рубль', 0.828010618686676),
 ('миллион_рубль', 0.8243743777275085),
 ('общий_сумма', 0.801665186882019),
 ('размер', 0.7922245264053345),
 ('тысяча_рубль', 0.7916114926338196),
 ('средний_размер', 0.7824811935424805),
 ('составлять_рубль', 0.7791329622268677),
 ('возмещение', 0.7769399881362915),
 ('размер_тысяча', 0.7733787298202515),
 ('региональный_бюджет', 0.7682029604911804)]

In [10]:
model.save("word2vec.model")

In [11]:
model.wv.most_similar('инфляция', topn=10)

[('темп_рост', 0.9167305827140808),
 ('курс_рубль', 0.9096757173538208),
 ('рост_экономика', 0.9013553261756897),
 ('ввп', 0.8933051228523254),
 ('рост_ввп', 0.8875564336776733),
 ('цена_нефть', 0.886102557182312),
 ('прогноз', 0.880277693271637),
 ('уровень_инфляция', 0.8788392543792725),
 ('ключевой_ставка', 0.878484308719635),
 ('безработица', 0.8775793313980103)]

In [12]:
model.wv.most_similar('навальный', topn=10)

[('алексей_навальный', 0.899869441986084),
 ('оппозиционер', 0.772693932056427),
 ('политзаключённый', 0.6728214621543884),
 ('навальный_отравить', 0.6484887003898621),
 ('навальный_отбывать', 0.6436409950256348),
 ('леонид_волков', 0.6417167782783508),
 ('пивоваров', 0.6317705512046814),
 ('оппозиционный_политика', 0.6289198994636536),
 ('правозащитник', 0.6281477808952332),
 ('любовь_соболь', 0.623923122882843)]